In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from torch import nn
import os
# from torch.utils.data import DataLoader
# from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split


In [ ]:
class Net_result(nn.Module):
    def __init__(self):
        super().__init__() # 2 x 10 x 240
        self.conv1 = nn.Conv2d(2, 6, (1,2),stride=(1,2)) # 6 x 10 x 120 
        self.pool = nn.MaxPool2d(2) # 6 x 5 x 60
        self.drop = nn.Dropout(p=0.15)
        self.fc1 = nn.Linear( 1800, 360)   
        self.fc2 = nn.Linear(360, 120)
        self.fc3 = nn.Linear(120, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x,1) 
        x = self.drop(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x